# Starting notebook

Let us visualize the data and produce low pass filtering

In [ ]:
# Some useful modules for notebooks
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
from tqdne.conf import DATASETDIR
import h5py
from scipy import signal
import numpy as np
from tqdm import tqdm
datapath = DATASETDIR / Path("wforms_GAN_input_v20220805.h5")


In [ ]:
from tqdne.dataset import compute_mean_std_features, extract_sample_from_h5file, features_keys, build_sample

means, stds = compute_mean_std_features(datapath, features_keys)

In [ ]:
# Opent the hdf5 file

sample_id = 0
with h5py.File(datapath, "r") as f:
    for key in f.keys():
        print(f"Key: {key} -- {f[key].shape}")

    time = f["time_vector"][:]
    waveform, features = extract_sample_from_h5file(f, sample_id)
plt.plot(time, waveform.T)
plt.xlabel("Time [s]")
features, stds, means

In [ ]:
fs = 100

In [ ]:
# plot the frequency response
b, a = signal.butter(N=2, Wn=[1., 15.], btype='bp', analog=True)
w, h = signal.freqs(b, a)
plt.semilogx(w, 20 * np.log10(abs(h)))
plt.title('Butterworth filter frequency response')
plt.xlabel('Frequency [radians / second]')
plt.ylabel('Amplitude [dB]')
plt.margins(0, 0.1)
plt.grid(which='both', axis='both')
plt.axvline(100, color='green') # cutoff frequency
plt.show()

In [ ]:

sos = signal.butter(2, 1, 'lp', fs=fs, output='sos')
for i, channel in enumerate(waveform):
    datafilt = signal.sosfilt(sos, channel)
    plt.figure(figsize=(10, 10))
    plt.subplot(len(waveform), 1, 1+i)
    plt.plot(time, datafilt,  'r', alpha=1,)
    plt.plot(time, channel , 'b',  alpha=0.5,)
    # plt.xlim(8, 12)


In [ ]:
# test filter on sinusoids
# t = np.linspace(-5, 50, fs*55, False)  # 55 second
# sig = np.sin(2*np.pi*0.25*t) + np.sin(2*np.pi*5*t)
# fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
# ax1.plot(t, sig)
# ax1.set_title('0.25 Hz and 5 Hz sinusoids')
# ax1.axis([-5, 50, -2, 2])

# sos = signal.butter(10, 1, 'lp', fs=fs, output='sos')
# filtered = signal.sosfilt(sos, sig)
# ax2.plot(t, filtered)
# ax2.set_title('After 15 Hz high-pass filter')
# ax1.axis([-5, 50, -2, 2])
# ax2.set_xlabel('Time [seconds]')
# plt.tight_layout()
# plt.show()

In [ ]:

sos = signal.butter(2, 1, 'lp', fs=fs, output='sos')

sample_id = 0
with h5py.File(datapath, "r") as f:
    time = f["time_vector"][:]
    waveform, features = extract_sample_from_h5file(f, sample_id)
    features, datafilt, waveform, scale = build_sample(waveform, features, means, stds, sos)

n_channels = waveform.shape[0]
plt.figure(figsize=(10, 10))
for i, (filt, orig) in enumerate(zip(datafilt, waveform)):
    plt.subplot(n_channels, 1, 1+i)
    plt.plot(time, filt,  'r', alpha=1,)
    plt.plot(time, orig , 'b',  alpha=0.5,)
    # plt.xlim(8, 12
features